In [2]:
CIFAR = '/content/drive/My Drive/Assignment/ML ASS 3/cifar-10-batches-py/'
import numpy as np

In [3]:
def unpickle(file):
    import pickle
    with open(file,'rb') as fo:
        cifar_dict = pickle.load(fo,encoding='bytes')
    return cifar_dict

In [4]:
dirs = ['batches.meta','data_batch_1','data_batch_2','data_batch_3','data_batch_4','data_batch_5','test_batch']

all_data = [0,1,2,3,4,5,6]

for i, direc in zip(all_data,dirs):
    all_data[i] = unpickle(CIFAR+direc)

In [5]:
batch_meta = all_data[0]
data_batch1 = all_data[1]
data_batch2 = all_data[2]
data_batch3 = all_data[3]
data_batch4 = all_data[4]
data_batch5 = all_data[5]
test_batch = all_data[6]

In [6]:
all_train_batches = [data_batch1,data_batch2,data_batch3,data_batch4,data_batch5]
test_batch1 = [test_batch]

training_image = np.vstack([d[b"data"] for d in all_train_batches])
train_len = len(training_image)
training_image = training_image.reshape(train_len,3*32*32)
training_label = np.hstack([d[b"labels"] for d in all_train_batches])

In [7]:
test_image = np.vstack([d[b"data"] for d in test_batch1])
test_len = len(test_image)
test_image = test_image.reshape(test_len,3*32*32)
test_label = np.hstack([d[b"labels"] for d in test_batch1])

In [8]:
##### RAJATA CODE for hog + color hist
import numpy as np
import matplotlib.colors as colors
from skimage.feature import hog
from skimage import color

X_train_hog = []

for img in training_image:

    img = np.reshape(img.T,(32,32,3))
    # color histogram
    array=np.asarray(img)
    arr=(array.astype(float))/255.0
    img_hsv = colors.rgb_to_hsv(arr[...,:3])
    img_color_hist = np.histogram(img_hsv[...,0],bins=10)
    hist_values = img_color_hist[0]

    #hog
    gray_image = color.rgb2gray(img)
    hog_feature = hog(gray_image,orientations=8, pixels_per_cell=(8, 8), cells_per_block=(1, 1))

    #joined
    X_train_hog.append(np.concatenate((hist_values,hog_feature),axis=None))

In [19]:
X_test_hog = []

for img in test_image:

    img = np.reshape(img.T,(32,32,3))
    # color histogram
    array=np.asarray(img)
    arr=(array.astype(float))/255.0
    img_hsv = colors.rgb_to_hsv(arr[...,:3])
    img_color_hist = np.histogram(img_hsv[...,0],bins=10)
    hist_values = img_color_hist[0]

    #hog
    gray_image = color.rgb2gray(img)
    hog_feature = hog(gray_image,orientations=8, pixels_per_cell=(8, 8), cells_per_block=(1, 1))

    #joined
    X_test_hog.append(np.concatenate((hist_values,hog_feature),axis=None))

In [20]:
import numpy
X_train =numpy.asarray(X_train_hog)
X_test = numpy.asarray(X_test_hog)

In [22]:
X_train.shape


(50000, 138)

In [23]:
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC 
import pickle 
# defining parameter range 
param_grid = {'C': [10],                 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,cv=5,n_jobs=-1) 
  
# fitting the model for grid search 
grid.fit(X_train, training_label) 

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 48.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [10], 'kernel': ['rbf']}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=3)

In [24]:
print(grid.best_params_) 

{'C': 10, 'kernel': 'rbf'}


In [25]:
print(grid.best_estimator_) 

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [31]:
sv = grid.best_estimator_.fit(X_train,training_label)

In [32]:
filename = 'SVMrbf10 HOG .sav'
pickle.dump(sv, open(filename, 'wb'))

In [27]:
grid_trainpredict = grid.predict(X_train)

In [28]:
grid_predictions = grid.predict(X_test) 

In [29]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(training_label, grid_trainpredict))

              precision    recall  f1-score   support

           0       0.35      0.38      0.37      5000
           1       0.22      0.33      0.26      5000
           2       0.29      0.02      0.04      5000
           3       0.28      0.04      0.07      5000
           4       0.28      0.25      0.26      5000
           5       0.25      0.36      0.30      5000
           6       0.32      0.34      0.33      5000
           7       0.25      0.33      0.28      5000
           8       0.28      0.33      0.31      5000
           9       0.19      0.26      0.22      5000

    accuracy                           0.26     50000
   macro avg       0.27      0.26      0.24     50000
weighted avg       0.27      0.26      0.24     50000



In [30]:
print(classification_report(test_label, grid_predictions))

              precision    recall  f1-score   support

           0       0.36      0.38      0.37      1000
           1       0.20      0.31      0.25      1000
           2       0.25      0.02      0.03      1000
           3       0.26      0.04      0.06      1000
           4       0.29      0.25      0.27      1000
           5       0.25      0.37      0.30      1000
           6       0.32      0.35      0.34      1000
           7       0.25      0.32      0.28      1000
           8       0.28      0.33      0.30      1000
           9       0.20      0.27      0.23      1000

    accuracy                           0.26     10000
   macro avg       0.27      0.26      0.24     10000
weighted avg       0.27      0.26      0.24     10000

